In [ ]:
import ipyvuetify as v
from sepal_ui import sepalwidgets as sw
from scripts.tile import *

In [ ]:
step2_content = v.StepperContent(
    key=1,
    step=1, 
    children=[step2()]
)  

step3_content = v.StepperContent(
    key=2,
    step=2, 
    children=[step3()]
)

step4_content = v.StepperContent(
    key=3,
    step=3, 
    children=[step4()]
)

step5_content = v.StepperContent(
    key=4,
    step=4, 
    children=[step5()]
)

stepper_children = [
    v.StepperStep(
        key=1,
        complete=False,
        step=1,
        editable=True,
        children=["Constraints"]
    ),
    v.Divider(),
    v.StepperStep(
        key=2,
        complete=False,
        step=2,
        editable=True,
        children=["Restoration potential and land use"]
    ),
    v.Divider(),
    v.StepperStep(
        key=3,
        complete=False,
        step=3,
        editable=True,
        children=["Restoration goals"]
    ),
    v.Divider(),
    v.StepperStep(
        key=4,
        complete=False,
        step=4,
        editable=True,
        children=["Restoration priorities"]
    ),
]

stepper_header = v.StepperHeader(
    children=stepper_children
)

aoi_content = v.Stepper(
        class_="mt-2",
         children=[
             stepper_header,
             v.StepperItems(
                 children=[
                     step2_content,
                     step3_content,
                     step4_content,
                     step5_content
                 ]
             )
         ]
)

In [ ]:
title = "Questionnaire Identifying Restoration Objective & Goals"
id_ = "questionnaire_widget"
process_tile = sw.Tile(id_, title, inputs=[aoi_content])
process_tile